- Preciso filtrar orders fulfillment
- Preciso filtrar orders aprovadas
- Preciso filtrar 'date_approved' em 'payments'. Se não houver desconsiderar


Em fulfillment aprovadas:
- Preciso saber a quantidade de vendas por id em determinado período (mesmo período da consulta ao estoque fulfillment, sempre findando no dia anterior a hoje)

In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
print(ACCESS_TOKEN)

APP_USR-6628000663056293-110915-d88565a275c6061fa574a2c979c280d7-233632476


## Verificando dados da conta

In [3]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {
  'Authorization': f'Bearer {ACCESS_TOKEN}'
}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

# Encontre posição do início e fim do campo "id"
start_pos = response.find('"id"')  # Encontra onde começa "id"
end_pos = response.find(',', start_pos)  # Encontra onde termina "id"

# Extraia o valor de "id"
seller_id = response[start_pos:end_pos].split(":")[1]
seller_id = seller_id.strip().strip('"')

print(f'O valor de "seller_id" é: {seller_id}')

{"id":233632476,"nickname":"MUSICALCENTERPG","registration_date":"2016-11-05T17:35:04.000-04:00","first_name":"MUSICAL CENTER PONTA","last_name":"GROSSA EIRELI -","gender":"","country_id":"BR","email":"musicalc168@gmail.com","identification":{"number":"10885874000127","type":"CNPJ"},"address":{"address":"Estrada da Gruta 775","city":"Louveira","state":"BR-SP","zip_code":"13290431"},"phone":{"area_code":"19","extension":"","number":"38787702","verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"brand","tags":["brand","large_seller","eshop","mshops","messages_as_seller"],"logo":null,"points":4515,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/MUSICALCENTERPG","seller_experience":"ADVANCED","bill_data":{"accept_credit_note":"Y"},"seller_reputation":{"level_id":"5_green","power_seller_status":"platinum","transactions":{"canceled":1502,"completed":32614,"period":"historic","ratings":{"negative":0.05,"neutral":0.03,"positive":0.92},"t

## Pegando dados de Vendas

A consulta a API do ML é feita com filtros para order.status = 'paid' e datas pré-determinadas

In [101]:
# date_from = '2023-10-10'
date_from = '2023-11-01'
date_to = '2023-11-09'

In [102]:
# URL base da API
base_url = "https://api.mercadolibre.com/orders/search"

# Parâmetros iniciais
params = {
    'seller': '233632476',
    # 'order.status': 'paid',
    'order.date_created.from': f'{date_from}T00:00:00.000-00:00',
    'order.date_created.to': f'{date_to}T00:00:00.000-00:00',
    'limit': 50,
    'offset': 0
}

# Token de autenticação
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

json_list = []

In [103]:
counter = 0

while True:
    response = requests.get(base_url, params=params, headers=headers)
    data = response.json()

    
    if 'results' in data:
        json_list.extend(data['results'])
    else:
        break
    
    # Verifique se há mais páginas
    if 'paging' in data:
        total_paging = data['paging'].get('total')
        if total_paging is None:
            break

        total_pages = math.ceil(total_paging / params['limit'])
        print(f'Total esperado de páginas: {counter}/{total_pages}')
        print(f'Offset atual: {params["offset"]}')

        counter += 1
        if params['offset'] >= total_paging:
            break

        params['offset'] += params['limit']
    else:
        break

print(f'Total de dados coletados: {len(json_list)}')

Total esperado de páginas: 0/24
Offset atual: 0
Total esperado de páginas: 1/24
Offset atual: 50
Total esperado de páginas: 2/24
Offset atual: 100
Total esperado de páginas: 3/24
Offset atual: 150
Total esperado de páginas: 4/24
Offset atual: 200
Total esperado de páginas: 5/24
Offset atual: 250
Total esperado de páginas: 6/24
Offset atual: 300
Total esperado de páginas: 7/24
Offset atual: 350
Total esperado de páginas: 8/24
Offset atual: 400
Total esperado de páginas: 9/24
Offset atual: 450
Total esperado de páginas: 10/24
Offset atual: 500
Total esperado de páginas: 11/24
Offset atual: 550
Total esperado de páginas: 12/24
Offset atual: 600
Total esperado de páginas: 13/24
Offset atual: 650
Total esperado de páginas: 14/24
Offset atual: 700
Total esperado de páginas: 15/24
Offset atual: 750
Total esperado de páginas: 16/24
Offset atual: 800
Total esperado de páginas: 17/24
Offset atual: 850
Total esperado de páginas: 18/24
Offset atual: 900
Total esperado de páginas: 19/24
Offset atua

In [ ]:
len(json_list)

In [80]:
df = pd.DataFrame(json_list)

# df.head(1)
df.columns


Index(['payments', 'fulfilled', 'taxes', 'order_request', 'expiration_date',
       'feedback', 'shipping', 'date_closed', 'id',
       'manufacturing_ending_date', 'order_items', 'date_last_updated',
       'last_updated', 'comment', 'pack_id', 'coupon', 'shipping_cost',
       'date_created', 'pickup_id', 'status_detail', 'tags', 'buyer', 'seller',
       'total_amount', 'paid_amount', 'currency_id', 'status', 'context',
       'mediations', 'cancel_detail'],
      dtype='object')

In [105]:
df1 = json_normalize(json_list, record_path='order_items', meta=['payments','fulfilled', 'date_closed', 'status', 'pack_id', 'id'])
df1.columns

Index(['quantity', 'unit_price', 'full_unit_price', 'currency_id',
       'manufacturing_days', 'picked_quantity', 'sale_fee', 'listing_type_id',
       'base_exchange_rate', 'base_currency_id', 'bundle', 'element_id',
       'item.id', 'item.title', 'item.category_id', 'item.variation_id',
       'item.seller_custom_field', 'item.global_price', 'item.net_weight',
       'item.variation_attributes', 'item.warranty', 'item.condition',
       'item.seller_sku', 'requested_quantity.measure',
       'requested_quantity.value', 'picked_quantity.measure',
       'picked_quantity.value', 'payments', 'fulfilled', 'date_closed',
       'status', 'pack_id', 'id'],
      dtype='object')

In [106]:
df1.head(1)

,quantity,unit_price,full_unit_price,currency_id,manufacturing_days,picked_quantity,sale_fee,listing_type_id,base_exchange_rate,base_currency_id,...,requested_quantity.measure,requested_quantity.value,picked_quantity.measure,picked_quantity.value,payments,fulfilled,date_closed,status,pack_id,id
0,1,239.2,299.0,BRL,None,NaN,39.47,gold_pro,None,None,...,unit,1,NaN,NaN,[{'reason': 'Ukulele Concert Acústico Tagima N...,True,2023-10-31T20:05:48.000-04:00,paid,2000004989794585,2000006791890642


In [107]:
cols = ['payments', 'item.id', 'item.title', 'item.variation_id', 'fulfilled', 'date_closed', 'status', 'id', 'pack_id']

In [108]:
df2 = df1[cols].copy()
df2.columns

Index(['payments', 'item.id', 'item.title', 'item.variation_id', 'fulfilled',
       'date_closed', 'status', 'id', 'pack_id'],
      dtype='object')

In [109]:
df2

,payments,item.id,item.title,item.variation_id,fulfilled,date_closed,status,id,pack_id
0,[{'reason': 'Ukulele Concert Acústico Tagima N...,MLB2171831134,Ukulele Concert Acústico Tagima Natural Mahoga...,1.742465e+11,True,2023-10-31T20:05:48.000-04:00,paid,2000006791890642,2000004989794585
1,[{'reason': 'Correia Basso Cla Peixe Guitarra ...,MLB3436292265,Correia Basso Cla Peixe Guitarra Violão Baixo ...,NaN,True,2023-10-31T20:11:32.000-04:00,paid,2000006791911116,None
2,[{'reason': 'Correia Para Guitarra Profissiona...,MLB3387432277,Correia Para Guitarra Profissional Polypro Ern...,NaN,True,2023-10-31T20:22:54.000-04:00,paid,2000006791995856,None
3,"[{'reason': 'Cabo Santo Angelo Ninja 4,57m P10...",MLB3829652726,"Cabo Santo Angelo Ninja 4,57m P10 Reto Guitarr...",NaN,True,2023-10-31T20:23:23.000-04:00,paid,2000006792003152,2000004989851393
4,[{'reason': 'Encordoamento Sao Goncalo Violao ...,MLB2154203848,Encordoamento Sao Goncalo Violao Nylon Cristal...,NaN,True,2023-10-31T20:26:56.000-04:00,paid,2000006792026842,2000004989863439
...,...,...,...,...,...,...,...,...,...
1193,[{'reason': 'Correia Guitarra Violão Baixo Alç...,MLB3780803056,Correia Guitarra Violão Baixo Alça Basso Cla09...,NaN,None,2023-11-08T20:46:36.000-04:00,paid,2000006850274832,2000005018888075
1194,[{'reason': 'Pandeiro Profissional Izzo Pele P...,MLB3427769549,Pandeiro Profissional Izzo Pele Preta 10 Corpo...,NaN,True,2023-11-08T20:59:31.000-04:00,paid,2000006850404126,None
1195,[{'reason': 'Jogo 5 Cordas 4ª Re Nylon Aço Pra...,MLB3201609925,Jogo 5 Cordas 4ª Re Nylon Aço Prata P/violao S...,NaN,None,2023-11-08T21:00:00.000-04:00,paid,2000006850373236,None
1196,[{'reason': 'Pandeiro Profissional Meia Lua Di...,MLB3900020030,Pandeiro Profissional Meia Lua Diversas Cores ...,NaN,True,2023-11-08T21:00:02.000-04:00,paid,2000006850362028,None


In [114]:
type(dfx)

pandas.core.series.Series

In [121]:
dfx = df1['payments']

# dfy = json_normalize(json_list, record_path='payments', meta=['fulfilled', 'date_closed', 'status', 'pack_id', 'id'])
dfy = json_normalize(json_list, record_path='payments', meta=['fulfilled', 'date_closed', 'pack_id'])
dfy

,reason,status_code,total_paid_amount,operation_type,transaction_amount,transaction_amount_refunded,date_approved,coupon_id,installments,authorization_code,...,order_id,currency_id,status,transaction_order_id,collector.id,atm_transfer_reference.transaction_id,atm_transfer_reference.company_id,fulfilled,date_closed,pack_id
0,Ukulele Concert Acústico Tagima Natural Mahoga...,None,239.20,regular_payment,239.2,0.0,2023-10-31T20:05:48.000-04:00,None,1,None,...,2000006791890642,BRL,approved,None,233632476,None,None,True,2023-10-31T20:05:48.000-04:00,2000004989794585
1,Correia Basso Cla Peixe Guitarra Violão Baixo ...,None,74.90,regular_payment,74.9,0.0,2023-10-31T20:10:31.000-04:00,None,1,None,...,2000006791911116,BRL,approved,None,233632476,None,1,True,2023-10-31T20:11:32.000-04:00,None
2,Correia Para Guitarra Profissional Polypro Ern...,None,87.82,regular_payment,75.9,0.0,2023-10-31T20:22:54.000-04:00,None,1,826318,...,2000006791995856,BRL,approved,None,233632476,None,None,True,2023-10-31T20:22:54.000-04:00,None
3,"Cabo Santo Angelo Ninja 4,57m P10 Reto Guitarr...",None,59.90,regular_payment,59.9,0.0,2023-10-31T20:23:22.000-04:00,None,1,None,...,2000006792003152,BRL,approved,None,233632476,None,None,True,2023-10-31T20:23:23.000-04:00,2000004989851393
4,Encordoamento Sao Goncalo Violao Nylon Cristal...,None,23.90,regular_payment,23.9,0.0,2023-10-31T20:26:55.000-04:00,None,1,None,...,2000006792026842,BRL,approved,None,233632476,None,None,True,2023-10-31T20:26:56.000-04:00,2000004989863439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1239,Cabo P10 X P10l Santo Angelo Ninja L 25ft - 7.62m,None,92.35,regular_payment,78.9,0.0,None,None,2,None,...,2000006848091944,BRL,rejected,None,233632476,None,None,None,2023-11-09T12:46:58.000-04:00,None
1240,Cabo P10 X P10l Santo Angelo Ninja L 25ft - 7.62m,None,92.35,regular_payment,78.9,0.0,None,None,1,None,...,2000006848091944,BRL,rejected,None,233632476,None,None,None,2023-11-09T12:46:58.000-04:00,None
1241,Cabo P10 X P10l Santo Angelo Ninja L 25ft - 7.62m,None,92.35,regular_payment,78.9,0.0,None,None,1,None,...,2000006848091944,BRL,rejected,None,233632476,None,None,None,2023-11-09T12:46:58.000-04:00,None
1242,Cabo P10 X P10l Santo Angelo Ninja L 25ft - 7.62m,None,92.35,regular_payment,78.9,0.0,None,None,2,None,...,2000006848091944,BRL,rejected,None,233632476,None,None,None,2023-11-09T12:46:58.000-04:00,None


In [96]:
x = df2['date_closed'].isnull
x

<bound method Series.isnull of 0       2023-10-31T20:05:48.000-04:00
1       2023-10-31T20:11:32.000-04:00
2       2023-10-31T20:22:54.000-04:00
3       2023-10-31T20:23:23.000-04:00
4       2023-10-31T20:26:56.000-04:00
                    ...              
1193    2023-11-08T20:46:36.000-04:00
1194    2023-11-08T20:59:31.000-04:00
1195    2023-11-08T21:00:00.000-04:00
1196    2023-11-08T21:00:02.000-04:00
1197    2023-11-09T12:46:58.000-04:00
Name: date_closed, Length: 1198, dtype: object>

In [97]:
df2['status'].value_counts()


status
paid         1171
cancelled      27
Name: count, dtype: int64

In [100]:
len(json_list)

1198

In [98]:
dfx = df2[df2['status'] == 'cancelled']
dfx

,item.id,item.title,item.variation_id,fulfilled,date_closed,status,id,pack_id
41,MLB1992889189,Kit 3 Pares De Baquetas D Bateria Liverpool Ni...,1.735513e+11,False,2023-11-01T07:14:59.000-04:00,cancelled,2000006793788290,None
89,MLB3427769549,Pandeiro Profissional Izzo Pele Preta 10 Corpo...,NaN,False,2023-11-01T13:28:20.000-04:00,cancelled,2000006796144556,2000004991929785
100,MLB1991060554,Correia Guitarra Violão Baixo Alça Basso Cla 0...,NaN,False,2023-11-01T15:42:50.000-04:00,cancelled,2000006797066012,None
224,MLB3901882078,"Cabo Pedal/pedal 6 (15,24 Cm) Plug L/l Pct C/3...",NaN,False,2023-11-02T19:50:17.000-04:00,cancelled,2000006805353722,2000004996576297
243,MLB1557018312,Ukulelê Soprano Profissional Tagima 21k Mahogany,5.789662e+10,False,2023-11-02T21:59:04.000-04:00,cancelled,2000006806198242,None
331,MLB3436515963,Correia Alça Classic Violão Guitarra Baixo Pre...,NaN,False,2023-11-03T13:22:46.000-04:00,cancelled,2000006809646050,None
405,MLB2075516288,Kit 6 Pares De Baquetas D Bateria Liverpool Ni...,1.738870e+11,False,2023-11-03T20:36:58.000-04:00,cancelled,2000006812391454,None
407,MLB1992889189,Kit 3 Pares De Baquetas D Bateria Liverpool Ni...,1.735513e+11,False,2023-11-03T20:43:02.000-04:00,cancelled,2000006812421532,None
413,MLB4023855954,Correia Para Violão Guitarra Contra Baixo Bass...,NaN,False,2023-11-03T21:10:17.000-04:00,cancelled,2000006812584390,2000005000221339
445,MLB3696018282,"2 Palheta Sax Alto Vandoren Java Origina 1 1,5...",1.785312e+11,False,2023-11-04T08:26:06.000-04:00,cancelled,2000006814092196,None


In [37]:
## Total de vendas de produtos ##

df3 = df2['item.id'].value_counts().reset_index()

df3.columns = ['item.id', 'count']

df3 = df3.sort_values(by='count', ascending=False)

df3

,item.id,count
0,MLB3435228689,441
2,MLB3434144067,360
1,MLB3427769549,360
3,MLB1992889189,289
4,MLB3829652726,203
...,...,...
428,MLB2977621147,1
427,MLB4145500762,1
426,MLB3492359265,1
425,MLB3493848849,1


In [38]:
x = 'MLB1992889189'
y = df2[df2['item.id'] == x]

y.head(3)

,item.id,item.title,item.variation_id,fulfilled,date_closed,status
38,MLB1992889189,Kit 3 Pares De Baquetas D Bateria Liverpool Ni...,1.735513e+11,True,2023-10-09T22:12:42.000-04:00,paid
43,MLB1992889189,Kit 3 Pares De Baquetas D Bateria Liverpool Ni...,1.735515e+11,True,2023-10-09T22:37:06.000-04:00,paid
79,MLB1992889189,Kit 3 Pares De Baquetas D Bateria Liverpool Ni...,1.735513e+11,True,2023-10-10T07:08:48.000-04:00,paid


## Pegando códigos para comparar os produtos que queremos

In [47]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)
df_codes.shape

C:\Users\couti\AppData\Local\Temp\ipykernel_20724\2578135159.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


(754, 5)

In [48]:
df_codes.columns
df3.columns

Index(['item.id', 'count'], dtype='object')

In [49]:
df_codes['ml_code'] = 'MLB' + df_codes['ml_code']
df_codes.head(1)

,tiny_id,ml_inventory_id,ml_code,created_at,updated_at
0,509517168,JFGN34621,MLB924922735,2023-11-07 15:46:15.284630,None


In [50]:
df3.head(1)

,item.id,count
0,MLB3435228689,441


## Comparando os dois DFs

In [54]:
df_filtered = df3.merge(df_codes, left_on='item.id', right_on='ml_code')
df_filtered['qtd_vendidos'] = df_filtered['count']
df_filtered = df_filtered.drop(['count', 'updated_at', 'created_at'], axis=1)
df_filtered

,item.id,tiny_id,ml_inventory_id,ml_code,qtd_vendidos
0,MLB1992889189,735947207,ELQK04205,MLB1992889189,289
1,MLB1992889189,698412654,FSNB97805,MLB1992889189,289
2,MLB1992889189,747848159,YXIW04349,MLB1992889189,289
3,MLB1992889189,698412654,TNXB03680,MLB1992889189,289
4,MLB1992889189,735947207,TNXB03680,MLB1992889189,289
...,...,...,...,...,...
241,MLB1629329134,598806000,HFNJ52229,MLB1629329134,1
242,MLB1440288810,693267243,AIRQ38676,MLB1440288810,1
243,MLB2000966185,729129329,ZJCO04233,MLB2000966185,1
244,MLB1813392401,512779533,JOGC18300,MLB1813392401,1
